In [1]:
pip install torch-pruning 

   ---------------------------------------- 0.0/60.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/60.2 kB ? eta -:--:--
   ------ --------------------------------- 10.2/60.2 kB ? eta -:--:--
   ------ --------------------------------- 10.2/60.2 kB ? eta -:--:--
   ------ --------------------------------- 10.2/60.2 kB ? eta -:--:--
   ------ --------------------------------- 10.2/60.2 kB ? eta -:--:--
   ------ --------------------------------- 10.2/60.2 kB ? eta -:--:--
   ------ --------------------------------- 10.2/60.2 kB ? eta -:--:--
   ---------------------------------- ----- 51.2/60.2 kB 124.9 kB/s eta 0:00:01
   ---------------------------------------- 60.2/60.2 kB 139.1 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Method
I will be using maginutde based pruning, the group of channel with lowes L2 norm will be pruned iteratively.
Layer-adaptive sparsity for the Magnitude-based Pruning: https://arxiv.org/abs/2010.07611



In [2]:
import torch 
import torch.nn as nn
import torchvision
from torchvision import transforms
from tqdm import tqdm
from torch.optim import lr_scheduler
import torch.optim as optim
import time
import os
import random
import shutil
import torch_pruning as tp

In [3]:
!pip install torchsummary

paper: https://arxiv.org/pdf/2305.14403.pdf

## Dataset Preperation 

In [4]:
from torch.utils.data import Dataset, DataLoader,SubsetRandomSampler
import numpy as np

from utils.wider_face import *
from utils.transform import transformations, val_transformations


batch_size = BATCH_SIZE =  64

train_data = WIDERFaceDataset(split='train',
                            transforms=transformations)

val_data = WIDERFaceDataset(split='val',
                            transforms=val_transformations)

train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle = True,
        num_workers=0,
        pin_memory=True,
        drop_last=False,
    )


val_size = len(val_data)
val_indices = np.random.choice(val_size, int(0.2 * val_size), replace=False)
val_sampler = SubsetRandomSampler(val_indices)
val_loader = DataLoader(val_data, sampler=val_sampler, batch_size=BATCH_SIZE, shuffle=False)

# val_loader = torch.utils.data.DataLoader(
#         val_data,

#         batch_size=batch_size,
#         shuffle = False,
#         num_workers=0,
#         pin_memory=True,
#         drop_last=False,
#     )

print('The ImageNet train set is ready. Size : {}'.format(len(train_loader)*batch_size))



dataloaders = dict()

dataloaders['train'] = train_loader
dataloaders['val'] = val_loader

dataset_sizes = {'train': len(train_loader)*batch_size,
                'val': len(val_loader)*batch_size}
print(dataset_sizes)

RuntimeError: Dataset not found or corrupted. You can use download=True to download and prepare it

In [ ]:
#for testing  the correct working of model and see if it can overfit train data or not 
import numpy as np
from torch.utils.data import DataLoader, SubsetRandomSampler
# Create indices for subset

train_size = len(train_data)
val_size = len(val_data)

train_indices = np.random.choice(train_size, int(0.4 * train_size), replace=False)
val_indices = np.random.choice(val_size, int(0.15 * val_size), replace=False)

# Create samplers
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

# Create DataLoaders
train_loader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_data, sampler=val_sampler, batch_size=BATCH_SIZE, shuffle=False)

# Assign to dataloaders dictionary
dataloaders = {
    'train': train_loader,
    'val': val_loader
}

dataset_sizes = {'train': len(train_loader)*batch_size,
                'val': len(val_loader)*batch_size}
print(dataset_sizes)

In [ ]:
from utils.utils import visualize_bb
from utils.wider_face import inverse_target

samples  =[]

for i in range(8,10):
# for i in train_indices[20:28]:
  img , targets = train_data[i]
  targets = [target.unsqueeze(0) for target in targets]
  bboxes, labels = inverse_target(targets)
#   print('L',labels )
#   print(bboxes, labels)
  sample = {'image': img, 'bbox':bboxes[1], 'labels':labels[1].long()}     #should plot properly for all S[i]
  samples+=[sample]


visualize_bb(samples)

## Model Defination and Loss function 

In [5]:
from model.yolov4 import TinyYoloV4
from model.loss import YoloV4_Loss
from model.train import *

In [7]:
print(f'Training \n')
torch.cuda.empty_cache()

model = TinyYoloV4(num_classes = 1)
model = torch.load('widerface_tinyyolov4.pth', map_location='cpu')   #using a model i trained before

model = model.to(device)

#loss function 
criterion = YoloV4_Loss()


# Observe that all parameters are being optimized
optimizer_ft = torch.optim.Adam(model.parameters(), lr=0.0001)

# Decay LR by a factor of 0.1 every 7 epochs
# scheduler = MultiStepLR(optimizer, milestones=[10,25,40, 60], gamma=0.1)
exp_lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer_ft, milestones=[60], gamma=0.1)

Training 



In [11]:
print(model.backbone.stage1)
print(model.backbone.stage2)
print(model.head)

Conv2dBatchLeaky (3, 32, kernel_size=3, stride=2, padding=1, negative_slope=0.1015625)
Conv2dBatchLeaky (32, 64, kernel_size=3, stride=2, padding=1, negative_slope=0.1015625)
Head(
  (out1): Sequential(
    (0): Conv2dBatchLeaky (512, 1024, kernel_size=3, stride=1, padding=1, negative_slope=0.1015625)
    (1): Conv2dBatchLeaky (1024, 18, kernel_size=1, stride=1, padding=0, negative_slope=0.1015625)
  )
  (out2): Sequential(
    (0): Conv2dBatchLeaky (256, 512, kernel_size=3, stride=1, padding=1, negative_slope=0.1015625)
    (1): Conv2dBatchLeaky (512, 18, kernel_size=1, stride=1, padding=0, negative_slope=0.1015625)
  )
)


In [14]:
def get_channel_sizes(model):
    channel_sizes = []

    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            channel_sizes.append(module.out_channels)
        
    print(channel_sizes)
    return channel_sizes

channel_sizes = get_channel_sizes(model)

[32, 64, 64, 128, 64, 64, 64, 64, 64, 128, 256, 128, 128, 128, 128, 128, 256, 512, 256, 256, 256, 256, 256, 512, 256, 512, 512, 256, 256, 256, 1024, 18, 512, 18]


# Defining Pruning

In [23]:
ignored_layers = []
ignored_layers += [model.backbone.stage1]
ignored_layers += [model.backbone.stage2]
ignored_layers += [model.backbone.stage3]
ignored_layers += [model.head]

In [24]:
print(ignored_layers)

[Conv2dBatchLeaky (3, 32, kernel_size=3, stride=2, padding=1, negative_slope=0.1015625), Conv2dBatchLeaky (32, 64, kernel_size=3, stride=2, padding=1, negative_slope=0.1015625), CSPBlock(
  (conv1): Conv2dBatchLeaky (64, 64, kernel_size=1, stride=1, padding=0, negative_slope=0.1015625)
  (conv2): Conv2dBatchLeaky (64, 128, kernel_size=3, stride=1, padding=1, negative_slope=0.1015625)
  (split0): Conv2dBatchLeaky (128, 64, kernel_size=1, stride=1, padding=0, negative_slope=0.1015625)
  (split1): Conv2dBatchLeaky (128, 64, kernel_size=1, stride=1, padding=0, negative_slope=0.1015625)
  (conv3): Conv2dBatchLeaky (64, 64, kernel_size=1, stride=1, padding=0, negative_slope=0.1015625)
  (conv4): Conv2dBatchLeaky (64, 64, kernel_size=3, stride=1, padding=1, negative_slope=0.1015625)
  (conv5): Conv2dBatchLeaky (64, 64, kernel_size=1, stride=1, padding=0, negative_slope=0.1015625)
), Head(
  (out1): Sequential(
    (0): Conv2dBatchLeaky (512, 1024, kernel_size=3, stride=1, padding=1, negative_

In [11]:
imp = tp.importance.LAMPImportance()

In [12]:
# 2. Pruner initialization
example_inputs = torch.randn(1, 3, 416, 416)
iterative_steps = 10 # You can prune your model to the target sparsity iteratively. #importance parameter
pruner = tp.pruner.MagnitudePruner(
    model, 
    example_inputs, 
    global_pruning=True, # If False, a uniform sparsity will be assigned to different layers.
    importance=imp, # importance criterion for parameter selection
    iterative_steps=iterative_steps, # the number of iterations to achieve target sparsity
    ch_sparsity=0.8, # Remove 80% channels, 
    ignored_layers=ignored_layers,
)


# Iterative Pruning

In [13]:
print('Started channel pruning   ')
base_macs, base_nparams = tp.utils.count_ops_and_params(model, example_inputs)

for i in range(iterative_steps):
    pruner.step()
    print(f'Iteration   {i+1} / {iterative_steps+1}---------------------------')
    macs, nparams = tp.utils.count_ops_and_params(model, example_inputs)
    print(model(example_inputs).shape)
    print(
        "  Iter %d/%d, Params: %.2f M => %.2f M"
        % (i+1, iterative_steps, base_nparams / 1e6, nparams / 1e6)
    )
    print(
        "  Iter %d/%d, MACs: %.2f G => %.2f G"
        % (i+1, iterative_steps, base_macs / 1e9, macs / 1e9)
    )
    
    print("="*16)
    # evaluate_model(model)
    ret_epoch = 3+i
    print(f'\n----------- Retraining for {ret_epoch} epochs-------------')
    
    
    model = model.to(device)

    #loss function 
    criterion = YoloV4_Loss()


    # Observe that all parameters are being optimized
    optimizer_ft = torch.optim.Adam(model.parameters(), lr=0.0001)

    # Decay LR by a factor of 0.1 every 7 epochs
    # scheduler = MultiStepLR(optimizer, milestones=[10,25,40, 60], gamma=0.1)
    exp_lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer_ft, milestones=[20], gamma=0.1)

    
    model, _ = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, dataloaders,
                    dataset_sizes, num_epochs= 3 + i )
    
    name= (80/10)*(i+1)
    torch.save(model, f'pruned_model_{name}%.pth')

Started channel pruning   
Iteration   1 / 11---------------------------
torch.Size([32, 10])
  Iter 1/10, Params: 20.19 M => 18.07 M
  Iter 1/10, MACs: 2.88 G => 2.80 G


100%|██████████| 313/313 [00:33<00:00,  9.25it/s]


Evaluation Loss: 0.5805 Acc: 0.8219

----------- Retraining for 3 epochs-------------
Epoch 1/3
----------


train Loss: 0.0847 Acc: 0.9702


val Loss: 0.1191 Acc: 0.9658

Epoch 2/3
----------


...

val Loss: 0.1592 Acc: 0.9537

Epoch 11/12
----------


train Loss: 0.0230 Acc: 0.9936


val Loss: 0.1542 Acc: 0.9565

Epoch 12/12
----------


train Loss: 0.0228 Acc: 0.9936


val Loss: 0.1548 Acc: 0.9573

Training complete in 53m 48s
Best val Acc: 0.957300


# Model Sizes

In [46]:
import os

file_size_bytes = os.path.getsize('/kaggle/input/pruning-effficientnet-v2-s-on-cifar10/best_model_9791.pt')
file_size_mb = file_size_bytes / (1024 * 1024)
print(f"Model file size before pruning: {file_size_mb:.2f} MB")

file_size_bytes = os.path.getsize('/kaggle/working/pruned_model_80.0%.pth')
file_size_mb = file_size_bytes / (1024 * 1024)
print(f"Model file size after pruning: {file_size_mb:.2f} MB")

Model file size before pruning: 77.99 MB
Model file size after pruning: 11.68 MB


# Measuring Inference time and model size


In [1]:
! pip install nn-meter
!nn-meter --list-predictors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 3.0 MB/s eta 0:00:00a 0:00:01
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/pkg_resources/__init__.py", line 3031, in _dep_map
    return self.__dep_map
  File "/opt/conda/lib/python3.10/site-packages/pkg_resources/__init__.py", line 2828, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/pkg_resources/__init__.py", line 3022, in _parsed_pkg_info
    return self._pkg_info
  File "/opt/conda/lib/python3.10/site-packages/pkg_resources/__init__.py", line 2828, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info. Did you mean: 'egg_info'?

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/bin/nn-meter", line 5,

In [9]:
!pip install onnx
!pip install onnx-simplifier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.3 MB/s eta 0:00:00a 0:00:01


In [5]:
# from nn_meter import load_latency_predictor
m_org = torch.load('/kaggle/input/pruning-effficientnet-v2-s-on-cifar10/best_model_9791.pt', map_location = torch.device('cpu')) #81% pruned model, unstructured
m_pruned = torch.load('/kaggle/working/pruned_model_87%.pth',  map_location = torch.device('cpu')) #original model

In [6]:
from nn_meter import load_latency_predictor

In [13]:
predictor = load_latency_predictor('cortexA76cpu_tflite21', 1.0)
lat_org = predictor.predict(m_org, 'torch')
lat_pruned = predictor.predict(m_pruned, 'torch')

(nn-Meter) checking local kernel predictors at /root/.nn_meter/data/predictor/cortexA76cpu_tflite21
(nn-Meter) load predictor /root/.nn_meter/data/predictor/cortexA76cpu_tflite21/avgpool.pkl
(nn-Meter) load predictor /root/.nn_meter/data/predictor/cortexA76cpu_tflite21/bn.pkl
(nn-Meter) load predictor /root/.nn_meter/data/predictor/cortexA76cpu_tflite21/concat.pkl


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.23.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.23.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


(nn-Meter) load predictor /root/.nn_meter/data/predictor/cortexA76cpu_tflite21/se.pkl
(nn-Meter) load predictor /root/.nn_meter/data/predictor/cortexA76cpu_tflite21/dwconv-bn-relu.pkl
(nn-Meter) load predictor /root/.nn_meter/data/predictor/cortexA76cpu_tflite21/relu.pkl
(nn-Meter) load predictor /root/.nn_meter/data/predictor/cortexA76cpu_tflite21/bnrelu.pkl
(nn-Meter) load predictor /root/.nn_meter/data/predictor/cortexA76cpu_tflite21/fc.pkl
(nn-Meter) load predictor /root/.nn_meter/data/predictor/cortexA76cpu_tflite21/hswish.pkl
(nn-Meter) load predictor /root/.nn_meter/data/predictor/cortexA76cpu_tflite21/add.pkl
(nn-Meter) load predictor /root/.nn_meter/data/predictor/cortexA76cpu_tflite21/maxpool.pkl
(nn-Meter) load predictor /root/.nn_meter/data/predictor/cortexA76cpu_tflite21/channelshuffle.pkl
(nn-Meter) load predictor /root/.nn_meter/data/predictor/cortexA76cpu_tflite21/conv-bn-relu.pkl
(nn-Meter) load predictor /root/.nn_meter/data/predictor/cortexA76cpu_tflite21/addrelu.pkl

In [14]:
print(f'Latency of original model: {lat_org}')
print(f'Latency of pruned model: {lat_pruned}')

Latency of original model: 328.95630089686364
Latency of pruned model: 128.62758512246805


In [35]:
import os
file_size_bytes = os.path.getsize('/kaggle/input/pruning-effficientnet-v2-s-on-cifar10/best_model_9791.pt')
file_size_mb = file_size_bytes / (1024 * 1024)
print(f"Model file size before pruning: {file_size_mb:.2f} MB")

file_size_bytes = os.path.getsize('/kaggle/working/pruned_model_87%.pth')
file_size_mb = file_size_bytes / (1024 * 1024)
print(f"Model file size after pruning: {file_size_mb:.2f} MB")

Model file size before pruning: 77.99 MB
Model file size after pruning: 5.66 MB
